In [166]:
%pip install pandas matplotlib
%pip install scikit-learn

You should consider upgrading via the '/Users/joelsng/Documents/GitHub/levelsliving-IMS/server/forecasting/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/joelsng/Documents/GitHub/levelsliving-IMS/server/forecasting/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [167]:
%pip install xgboost==2.0.3

You should consider upgrading via the '/Users/joelsng/Documents/GitHub/levelsliving-IMS/server/forecasting/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [168]:
import pandas as pd
import numpy as np
import re, json
from pathlib import Path
import datetime
import xgboost as xgb
import numpy as np
import pandas as pd
import json

In [169]:
df = pd.read_csv("sampleData/master_orders.csv")
df["Order Date"] = pd.to_datetime(df["Order Date"], utc=True)
df = df.sort_values("Order Date")

In [170]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 17684 entries, 17563 to 1685
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   Order Date     17506 non-null  datetime64[ns, UTC]
 1   SKU            16507 non-null  object             
 2   Item           17683 non-null  object             
 3   Variant        14338 non-null  object             
 4   Quantity       17663 non-null  float64            
 5   Delivery Date  11825 non-null  object             
 6   Delivery Time  11573 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(1), object(5)
memory usage: 1.1+ MB
None


# Aggregate based on monthly

In [171]:
monthly = (
    df
    .set_index("Order Date")
    .groupby(["SKU", pd.Grouper(freq="ME")])["Quantity"]
    .sum()
    .reset_index()
    .rename(columns={"Order Date": "Month"})
    .sort_values(["SKU", "Month"])
)

In [172]:
monthly.groupby("Month")["Quantity"].sum().reset_index().sort_values("Month")

,Month,Quantity
0,2022-04-30 00:00:00+00:00,91.0
1,2022-05-31 00:00:00+00:00,280.0
2,2022-06-30 00:00:00+00:00,339.0
3,2022-07-31 00:00:00+00:00,431.0
4,2022-08-31 00:00:00+00:00,392.0
5,2022-09-30 00:00:00+00:00,534.0
6,2022-10-31 00:00:00+00:00,412.0
7,2022-11-30 00:00:00+00:00,704.0
8,2022-12-31 00:00:00+00:00,720.0
9,2023-01-31 00:00:00+00:00,502.0


# Get top 5 SKUs

In [173]:
top_k = 5

In [174]:
vol_df = (
    monthly.groupby("SKU", observed=True)["Quantity"]
    .sum()
    .rename("total_qty")
    .reset_index()
)

vol_df = vol_df.sort_values(["total_qty", "SKU"], ascending=[False, True], kind="mergesort").reset_index(drop=True)

vol_df.head()

,SKU,total_qty
0,SF50,5108.0
1,CU-1,1423.0
2,SC5,674.0
3,DR50,592.0
4,SC15,538.0


In [175]:
top5 = vol_df.head(top_k)["SKU"].tolist()
top5

['SF50', 'CU-1', 'SC5', 'DR50', 'SC15']

# Stratify SKUs into sales volume

In [176]:
rest_df = vol_df[~vol_df["SKU"].isin(top5)].copy()
rest_total = rest_df["total_qty"].sum()

rest_df["cum_share"] = rest_df["total_qty"].cumsum() / rest_total


In [177]:
bins   = [-np.inf, 0.70, 0.95, np.inf]
labels = ["highest", "medium", "rest"]

rest_tiers = pd.cut(
    rest_df["cum_share"],
    bins=bins,
    labels=labels,
    include_lowest=True
).astype("string")

In [178]:
solo_labels = pd.Series({sku: f"solo::{sku}"} for sku in top5)
solo_labels = pd.Series({sku: f"solo::{sku}" for sku in top5}, name="tier", dtype="string")

rest_labels = pd.Series(rest_tiers.values, index=rest_df["SKU"].values, name="tier", dtype="string")

sku_tier = pd.concat([solo_labels, rest_labels])
sku_tier = sku_tier.astype("string")

sku_tier.name = "tier"


In [179]:
tiered = monthly.merge(sku_tier.to_frame(), left_on="SKU", right_index=True, how="inner")


In [180]:
print(sku_tier.value_counts())
print((sku_tier.value_counts(normalize=True) * 100).round(2).astype(str) + "%")

tier
rest          633
medium        613
highest        87
solo::SF50      1
solo::CU-1      1
solo::SC5       1
solo::DR50      1
solo::SC15      1
Name: count, dtype: Int64
tier
rest          47.31%
medium        45.81%
highest         6.5%
solo::SF50     0.07%
solo::CU-1     0.07%
solo::SC5      0.07%
solo::DR50     0.07%
solo::SC15     0.07%
Name: proportion, dtype: object


In [181]:
tiered

,SKU,Month,Quantity,tier
0,10031400,2022-08-31 00:00:00+00:00,1.0,medium
1,10031400,2022-10-31 00:00:00+00:00,1.0,medium
2,10031737,2022-09-30 00:00:00+00:00,1.0,medium
3,10031737,2022-12-31 00:00:00+00:00,1.0,medium
4,10057207,2022-06-30 00:00:00+00:00,1.0,medium
...,...,...,...,...
3775,Zara/Storage/Single,2025-06-30 00:00:00+00:00,1.0,rest
3776,Zara/Storage/Single,2025-07-31 00:00:00+00:00,1.0,rest
3777,Zara/Storage/SuperSingle,2025-04-30 00:00:00+00:00,1.0,rest
3778,Zara/Storage/SuperSingle,2025-05-31 00:00:00+00:00,1.0,rest


In [182]:
df_tier_month = (
    tiered.assign(Month=pd.to_datetime(tiered["Month"]).dt.to_period("M").dt.to_timestamp())
      .groupby(["tier", "Month"], as_index=False)["Quantity"]
      .sum()
      .sort_values(["tier", "Month"])
)

df_tier_month

/var/folders/tk/ptln5ljx66g1yqhg_dd36_200000gn/T/ipykernel_17665/1324460882.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  tiered.assign(Month=pd.to_datetime(tiered["Month"]).dt.to_period("M").dt.to_timestamp())


,tier,Month,Quantity
0,highest,2022-04-01,18.0
1,highest,2022-05-01,143.0
2,highest,2022-06-01,202.0
3,highest,2022-07-01,271.0
4,highest,2022-08-01,285.0
...,...,...,...
260,solo::SF50,2024-08-01,104.0
261,solo::SF50,2024-09-01,106.0
262,solo::SF50,2024-10-01,4.0
263,solo::SF50,2025-03-01,2.0


# Feature Engineering

In [183]:
df_tier_month["month_num"] = df_tier_month["Month"].dt.month
df_tier_month["year"] = df_tier_month["Month"].dt.year

# demand lag by x months
df_tier_month = df_tier_month.sort_values(["tier", "Month"])
df_tier_month["lag_1"] = df_tier_month.groupby("tier")["Quantity"].shift(1)
df_tier_month["lag_2"] = df_tier_month.groupby("tier")["Quantity"].shift(2)
df_tier_month["lag_3"] = df_tier_month.groupby("tier")["Quantity"].shift(3)

# month sin/cos seasonality
df_tier_month["month_sin"] = np.sin(2 * np.pi * df_tier_month["month_num"] / 12)
df_tier_month["month_cos"] = np.cos(2 * np.pi * df_tier_month["month_num"] / 12)


In [184]:
# use log_qty as target to train
df_tier_month["log_qty"] = np.log1p(df_tier_month["Quantity"])

In [185]:
df_tier_month

,tier,Month,Quantity,month_num,year,lag_1,lag_2,lag_3,month_sin,month_cos,log_qty
0,highest,2022-04-01,18.0,4,2022,NaN,NaN,NaN,8.660254e-01,-5.000000e-01,2.944439
1,highest,2022-05-01,143.0,5,2022,18.0,NaN,NaN,5.000000e-01,-8.660254e-01,4.969813
2,highest,2022-06-01,202.0,6,2022,143.0,18.0,NaN,1.224647e-16,-1.000000e+00,5.313206
3,highest,2022-07-01,271.0,7,2022,202.0,143.0,18.0,-5.000000e-01,-8.660254e-01,5.605802
4,highest,2022-08-01,285.0,8,2022,271.0,202.0,143.0,-8.660254e-01,-5.000000e-01,5.655992
...,...,...,...,...,...,...,...,...,...,...,...
260,solo::SF50,2024-08-01,104.0,8,2024,85.0,50.0,92.0,-8.660254e-01,-5.000000e-01,4.653960
261,solo::SF50,2024-09-01,106.0,9,2024,104.0,85.0,50.0,-1.000000e+00,-1.836970e-16,4.672829
262,solo::SF50,2024-10-01,4.0,10,2024,106.0,104.0,85.0,-8.660254e-01,5.000000e-01,1.609438
263,solo::SF50,2025-03-01,2.0,3,2025,4.0,106.0,104.0,1.000000e+00,6.123234e-17,1.098612


# Training and Evaluation

In [186]:
RANDOM_SEED = 42
N_ESTIMATORS = 500
EARLY_STOPPING = 50

In [187]:
def train_xgb_core_api(
    df_tier,
    # feat=("month_num", "year", "sku_id", "lag_1", "lag_2", "lag_3", "month_sin", "month_cos"),
    feat=("month_num", "year", "lag_1", "lag_2", "lag_3", "month_sin", "month_cos"),
):
    # Drop rows that cannot form lag_1
    df_tier = df_tier.dropna(subset=["lag_1"]).reset_index(drop=True)

    # If too small, return 6-tuple with features so callers can unpack safely
    if len(df_tier) < 20:
        return None, None, None, None, None, list(feat)

    X = df_tier[list(feat)].to_numpy(dtype=float)
    y = df_tier["log_qty"].to_numpy(dtype=float)

    # 80/20 split with guards so validation is never empty
    split_idx = int(len(df_tier) * 0.8)
    if split_idx <= 0:
        split_idx = 1
    if split_idx >= len(df_tier):
        split_idx = len(df_tier) - 1

    X_tr, X_va = X[:split_idx], X[split_idx:]
    y_tr, y_va = y[:split_idx], y[split_idx:]

    dtr = xgb.DMatrix(X_tr, label=y_tr, feature_names=list(feat))
    dva = xgb.DMatrix(X_va, label=y_va, feature_names=list(feat))

    params = {
        "objective": "reg:squarederror",
        "eta": 0.1,
        "max_depth": 4,
        "subsample": 0.9,
        "colsample_bytree": 0.9,
        "eval_metric": ["rmse", "mae"],
        "seed": 42,
    }
    ev = [(dtr, "train"), (dva, "valid")]
    bst = xgb.train(
        params,
        dtr,
        num_boost_round=N_ESTIMATORS,
        evals=ev,
        early_stopping_rounds=EARLY_STOPPING,
        verbose_eval=False,
    )

    # Predict using best iteration if available
    best_iter = getattr(bst, "best_iteration", None)
    if best_iter is None:
        pred = bst.predict(dva)
    else:
        pred = bst.predict(dva, iteration_range=(0, best_iter + 1))

    mae = float(np.mean(np.abs(y_va - pred)))
    rmse = float(np.sqrt(np.mean((y_va - pred) ** 2)))
    mape = float(np.mean(np.abs((y_va - pred) / np.clip(y_va, 1e-8, None))) * 100)
    smape = float(100 * np.mean(2 * np.abs(y_va - pred) / (np.abs(y_va) + np.abs(pred) + 1e-8)))

    return bst, mae, rmse, mape, smape, list(feat)


In [188]:
def slugify_tier(tier: str) -> str:
    s = re.sub(r"[^A-Za-z0-9]+", "_", str(tier)).strip("_")
    return s or "tier"

In [189]:
models = {}
tier_metrics = {}
report_rows = []
future_all = []

cur_dir = Path(__file__).resolve().parents[1] if "__file__" in globals() else Path.cwd()
save_dir = cur_dir / "models"
save_dir.mkdir(parents=True, exist_ok=True)

tier_list = df_tier_month["tier"].dropna().astype(str).unique().tolist()

for tier in tier_list:
    sub = df_tier_month.loc[df_tier_month["tier"] == tier].copy()
    if sub.empty:
        print(f"[{tier}] skipped (no rows)")
        continue

    bst, mae, rmse, mape, smape, feat = train_xgb_core_api(sub)
    if bst is None:
        print(f"[{tier}] skipped (not enough data)")
        continue

    # Stable, safe filename per tier (e.g., solo__SKU_123)
    tier_slug = slugify_tier(tier)
    model_fname = f"xgb_sku_{tier_slug}_model.ubj"
    model_path = save_dir / model_fname
    bst.save_model(model_path)
    models[tier] = (bst, feat)

    # best_iteration can be None if early stopping not triggered
    best_iter = getattr(bst, "best_iteration", None)
    if best_iter is None:
        print(f"[{tier}] MAE={mae:.3f}  RMSE={rmse:.3f}  MAPE={mape:.2f}%  SMAPE={smape:.2f}%")
    else:
        print(f"[{tier}] MAE={mae:.3f}  RMSE={rmse:.3f}  MAPE={mape:.2f}%  SMAPE={smape:.2f}%  (best_iter={best_iter})")

    # ---- NEW: collect metrics for meta/report ----
    tier_metrics[tier] = {
        "model_path": model_fname,
        "features": list(feat),
        "mae": float(mae),
        "rmse": float(rmse),
        "mape_pct": float(mape),
        "smape_pct": float(smape),
        "best_iter": int(best_iter) if best_iter is not None else None,
        "notes": "Tiered SKU forecaster; 'solo::SKU' denotes per-SKU model."
    }
    report_rows.append({
        "tier": tier,
        "model_path": model_fname,
        "mae": float(mae),
        "rmse": float(rmse),
        "mape_pct": float(mape),
        "smape_pct": float(smape),
        "best_iter": int(best_iter) if best_iter is not None else None,
    })

    # ---------- Build next-month features per SKU in this tier ----------
    tier_series = (
        sub.loc[:, ["tier", "Month", "Quantity"]]
        .assign(Month=pd.to_datetime(sub["Month"]).dt.to_period("M").dt.to_timestamp())
        .groupby(["tier", "Month"], as_index=False)["Quantity"].sum()
        .sort_values(["tier", "Month"])
    )

    # lags over the TIER series
    lag_1 = tier_series["Quantity"].iloc[-1] if len(tier_series) >= 1 else np.nan
    lag_2 = tier_series["Quantity"].iloc[-2] if len(tier_series) >= 2 else np.nan
    lag_3 = tier_series["Quantity"].iloc[-3] if len(tier_series) >= 3 else np.nan

    fut_month = tier_series["Month"].iloc[-1] + pd.offsets.MonthEnd(1)
    fut_month_num = fut_month.month

    future = pd.DataFrame({
        "month_num": [fut_month_num],
        "year": [fut_month.year],
        "lag_1": [lag_1],
        "lag_2": [lag_2],
        "lag_3": [lag_3],
        "month_sin": [np.sin(2 * np.pi * fut_month_num / 12)],
        "month_cos": [np.cos(2 * np.pi * fut_month_num / 12)],
    })

    # Ensure exact feature order expected by this tier’s model
    # Ensure the feature matrix matches the model's expected columns and order
    for c in feat:
        if c not in future.columns:
            future[c] = np.nan
    X_future = future[list(feat)].to_numpy(dtype=float)

    dfut = xgb.DMatrix(X_future, feature_names=list(feat))

    preds_log = (
        models[tier][0].predict(dfut)
        if best_iter is None
        else models[tier][0].predict(dfut, iteration_range=(0, best_iter + 1))
    )
    preds = np.expm1(preds_log)  # inverse of log1p

    # One row per TIER only (no SKU column here)
    out = pd.DataFrame({
        "tier": [tier],
        "Month": [fut_month],
        "predicted_qty": preds
    })
    future_all.append(out)


future_preds = pd.concat(future_all, ignore_index=True)
print(future_preds.head(20))


[highest] MAE=0.487  RMSE=0.681  MAPE=11.72%  SMAPE=10.73%  (best_iter=61)
[medium] MAE=0.679  RMSE=0.772  MAPE=15.05%  SMAPE=14.67%  (best_iter=163)
[rest] MAE=0.545  RMSE=0.757  MAPE=18.38%  SMAPE=15.50%  (best_iter=34)
[solo::CU-1] MAE=0.459  RMSE=0.566  MAPE=12.52%  SMAPE=12.58%  (best_iter=12)
[solo::DR50] skipped (not enough data)
[solo::SC15] MAE=1.050  RMSE=1.277  MAPE=122.68%  SMAPE=60.00%  (best_iter=14)
[solo::SC5] MAE=0.813  RMSE=0.876  MAPE=46.80%  SMAPE=36.19%  (best_iter=38)
[solo::SF50] skipped (not enough data)
         tier      Month  predicted_qty
0     highest 2025-09-30      55.516361
1      medium 2025-09-30     147.967102
2        rest 2025-09-30      10.008191
3  solo::CU-1 2025-09-30      40.154716
4  solo::SC15 2025-09-30       7.828911
5   solo::SC5 2025-09-30      12.425640


In [190]:
# writing metadata artifacts
# models_meta.json
models_meta = {}
for t, info in tier_metrics.items():
    models_meta[t] = {
        "model_path": info["model_path"],
        "features": info["features"],
        "metrics": {
            "mae": info["mae"],
            "rmse": info["rmse"],
            "mape_pct": info["mape_pct"],
            "smape_pct": info["smape_pct"],
            "best_iter": info["best_iter"],
        },
        "notes": info["notes"],
    }

# # embed training meta
# models_meta["_training_context"] = {
#     "created_at": datetime.datetime.utcnow().isoformat() + "Z",
#     "xgboost_version": xgb.__version__,
# }

# with open(save_dir / "models_meta.json", "w") as f:
#     json.dump(models_meta, f, indent=2)
# print("Wrote:", save_dir / "models_meta.json")

# # sku_tier_map.csv & sku_tier_map.json (SKU → tier used)
# # Expect sku_tier: pd.Series where index=SKU, value=tier
# sku_tier_path_csv = save_dir / "sku_tier_map.csv"
# sku_tier_path_json = save_dir / "sku_tier_map.json"

# sku_tier.to_frame().reset_index().rename(columns={"index": "SKU"}).to_csv(sku_tier_path_csv, index=False)
# sku_tier.to_frame().reset_index().rename(columns={"index": "SKU"}).to_json(sku_tier_path_json, orient="records")
# print("Wrote:", sku_tier_path_csv)
# print("Wrote:", sku_tier_path_json)

# # human-readable report
# pd.DataFrame(report_rows).sort_values("tier").to_csv(save_dir / "models_report.csv", index=False)
# print("Wrote:", save_dir / "models_report.csv")

# Create an Inventory level model

In [191]:
# params
RANDOM_SEED = 42
N_ESTIMATORS = 500
EARLY_STOPPING = 50
MODEL_NAME = "xgb_inventory_model.ubj"   # single pooled model
ARTIFACT_DIR = (Path.cwd() / "models")  # adjust if needed
ARTIFACT_DIR.mkdir(parents=True, exist_ok=True)

In [192]:
inventory_monthly = (
    monthly
    .groupby("Month")["Quantity"]
    .sum()
    .reset_index()
    .sort_values("Month")
    .reset_index(drop=True)
)

In [193]:
# Lag features
inventory_monthly["lag_1"] = inventory_monthly["Quantity"].shift(1)
inventory_monthly["lag_2"] = inventory_monthly["Quantity"].shift(2)
inventory_monthly["lag_3"] = inventory_monthly["Quantity"].shift(3)

# Seasonality
inventory_monthly["month_num"] = inventory_monthly["Month"].dt.month
inventory_monthly["year"] = inventory_monthly["Month"].dt.year
inventory_monthly["month_sin"] = np.sin(2 * np.pi * inventory_monthly["month_num"] / 12)
inventory_monthly["month_cos"] = np.cos(2 * np.pi * inventory_monthly["month_num"] / 12)

inventory_monthly["log_qty"] = np.log1p(inventory_monthly["Quantity"])

# Drop first few rows without lag features
# train_df = inventory_monthly.dropna(subset=["lag_1"]).reset_index(drop=True)

In [194]:
# all_months = sorted(train_df["Month"].unique())
# cut_idx = max(1, int(len(all_months) * 0.8))
# cutoff = all_months[cut_idx - 1]

# is_train = train_df["Month"] <= cutoff
# is_valid = train_df["Month"] > cutoff

# tr = train_df.loc[is_train]
# va = train_df.loc[is_valid]

In [195]:
# FEATURES = ["month_num", "year", "lag_1", "lag_2", "lag_3", "month_sin", "month_cos"]
# TARGET = "log_qty"

# dtr = xgb.DMatrix(tr[FEATURES].to_numpy(dtype=float), label=tr[TARGET], feature_names=FEATURES)
# dva = xgb.DMatrix(va[FEATURES].to_numpy(dtype=float), label=va[TARGET], feature_names=FEATURES)

# params = {
#     "objective": "reg:squarederror",
#     "eta": 0.10,
#     "max_depth": 6,
#     "eval_metric": ["rmse","mae"],
#     "seed": 42,
# }

# evals = [(dtr, "train"), (dva, "valid")]
# bst = xgb.train(params, 
#                 dtr, 
#                 num_boost_round=N_ESTIMATORS,
#                 evals=evals,
#                 early_stopping_rounds=EARLY_STOPPING,
#                 verbose_eval=False)

# if best_iter is None:
#     pred = bst.predict(dva)
# else:
#     pred = bst.predict(dva, iteration_range=(0, best_iter + 1))

# y_true = va[TARGET].to_numpy(dtype=float)
# y_pred = pred.astype(float)

# mae = float(np.mean(np.abs(y_true - y_pred)))
# rmse = float(np.sqrt(np.mean((y_true - y_pred) ** 2)))

# # MAPE (%). Avoid division by zero by clipping denominator.
# mape = float(np.mean(np.abs((y_true - y_pred) / np.clip(y_true, 1e-8, None))) * 100)

# # SMAPE (%). Symmetric MAPE recommended for intermittent/low demand.
# smape = float(100 * np.mean(2 * np.abs(y_true - y_pred) /
#                             (np.abs(y_true) + np.abs(y_pred) + 1e-8)))

# print(f"[INVENTORY MODEL] MAE={mae:.3f}  RMSE={rmse:.3f}  "
#       f"MAPE={mape:.2f}%  SMAPE={smape:.2f}%"
#       + ("" if best_iter is None else f"  (best_iter={best_iter})"))


In [196]:
# last = train_df.sort_values("Month").iloc[-1]
# fut_month = last["Month"] + pd.offsets.MonthEnd(1)
# fut_m = fut_month.month

# row = {
#     "month_num": fut_m,
#     "year": fut_month.year,
#     "lag_1": last["Quantity"],
#     "lag_2": last.get("lag_1", np.nan),
#     "lag_3": last.get("lag_2", np.nan),
#     "month_sin": np.sin(2 * np.pi * fut_m / 12),
#     "month_cos": np.cos(2 * np.pi * fut_m / 12),
# }

# X = np.array([[row[f] for f in FEATURES]], dtype=float)
# dfut = xgb.DMatrix(X, feature_names=FEATURES)

# inv_forecast = float(bst.predict(dfut)[0])
# print(f"Next-month inventory forecast: {inv_forecast:.2f}")


In [197]:
# model_path = ARTIFACT_DIR / "xgb_inventory_pooled_model.json"
# bst.save_model(str(model_path))

# meta = {
#     "inventory_pooled": {
#         "model_path": model_path.name,
#         "features": FEATURES,
#         "notes": "Aggregated inventory forecaster (no SKU input)"
#     }
# }
# with open(ARTIFACT_DIR / "inventory_models_meta.json", "w") as f:
#     json.dump(meta, f, indent=2)

# print("Saved inventory model and meta files.")


In [198]:
inventory_monthly

,Month,Quantity,lag_1,lag_2,lag_3,month_num,year,month_sin,month_cos,log_qty
0,2022-04-30 00:00:00+00:00,91.0,NaN,NaN,NaN,4,2022,8.660254e-01,-5.000000e-01,4.521789
1,2022-05-31 00:00:00+00:00,280.0,91.0,NaN,NaN,5,2022,5.000000e-01,-8.660254e-01,5.638355
2,2022-06-30 00:00:00+00:00,339.0,280.0,91.0,NaN,6,2022,1.224647e-16,-1.000000e+00,5.828946
3,2022-07-31 00:00:00+00:00,431.0,339.0,280.0,91.0,7,2022,-5.000000e-01,-8.660254e-01,6.068426
4,2022-08-31 00:00:00+00:00,392.0,431.0,339.0,280.0,8,2022,-8.660254e-01,-5.000000e-01,5.973810
5,2022-09-30 00:00:00+00:00,534.0,392.0,431.0,339.0,9,2022,-1.000000e+00,-1.836970e-16,6.282267
6,2022-10-31 00:00:00+00:00,412.0,534.0,392.0,431.0,10,2022,-8.660254e-01,5.000000e-01,6.023448
7,2022-11-30 00:00:00+00:00,704.0,412.0,534.0,392.0,11,2022,-5.000000e-01,8.660254e-01,6.558198
8,2022-12-31 00:00:00+00:00,720.0,704.0,412.0,534.0,12,2022,-2.449294e-16,1.000000e+00,6.580639
9,2023-01-31 00:00:00+00:00,502.0,720.0,704.0,412.0,1,2023,5.000000e-01,8.660254e-01,6.220590


In [199]:
bst, mae, rmse, mape, smape, feat = train_xgb_core_api(inventory_monthly)

In [200]:
# Stable, safe filename
model_fname = MODEL_NAME
model_path = save_dir / model_fname
bst.save_model(model_path)
models["inventory"] = (bst, feat)

best_iter = getattr(bst, "best_iteration", None)
if best_iter is None:
    print(f"[inventory] MAE={mae:.3f}  RMSE={rmse:.3f}  MAPE={mape:.2f}%  SMAPE={smape:.2f}%")
else:
    print(f"[inventory] MAE={mae:.3f}  RMSE={rmse:.3f}  MAPE={mape:.2f}%  SMAPE={smape:.2f}%  (best_iter={best_iter})")

tier_metrics["inventory"] = {
    "model_path": model_fname,
    "features": list(feat),
    "mae": float(mae),
    "rmse": float(rmse),
    "mape_pct": float(mape),
    "smape_pct": float(smape),
    "best_iter": int(best_iter) if best_iter is not None else None,
    "notes": "Inventory overall level forecaster."
}
report_rows.append({
    "tier": "inventory",
    "model_path": model_fname,
    "mae": float(mae),
    "rmse": float(rmse),
    "mape_pct": float(mape),
    "smape_pct": float(smape),
    "best_iter": int(best_iter) if best_iter is not None else None,
})

[inventory] MAE=0.421  RMSE=0.575  MAPE=7.79%  SMAPE=7.39%  (best_iter=50)


In [201]:
last = inventory_monthly.sort_values("Month").iloc[-1]
fut_month = last["Month"] + pd.offsets.MonthEnd(1)
fut_m = fut_month.month

row = {
    "month_num": fut_m,
    "year": fut_month.year,
    "lag_1": last["Quantity"],
    "lag_2": last.get("lag_1", np.nan),
    "lag_3": last.get("lag_2", np.nan),
    "month_sin": np.sin(2 * np.pi * fut_m / 12),
    "month_cos": np.cos(2 * np.pi * fut_m / 12),
}

X = np.array([[row[f] for f in FEATURES]], dtype=float)
dfut = xgb.DMatrix(X, feature_names=FEATURES)

inv_forecast_log = float(bst.predict(dfut)[0])
inv_forecast = float(np.expm1(inv_forecast_log))

print(f"Next-month inventory forecast: {inv_forecast:.2f}")


Next-month inventory forecast: 280.49


In [202]:
for t, info in tier_metrics.items():
    print(t)

highest
medium
rest
solo::CU-1
solo::SC15
solo::SC5
inventory


In [203]:
for t, info in tier_metrics.items():
    if t == 'inventory':
        models_meta[t] = {
            "model_path": info["model_path"],
            "features": info["features"],
            "metrics": {
                "mae": info["mae"],
                "rmse": info["rmse"],
                "mape_pct": info["mape_pct"],
                "smape_pct": info["smape_pct"],
                "best_iter": info["best_iter"],
            },
            "notes": info["notes"],
        }

In [204]:
# embed training meta
models_meta["_training_context"] = {
    "created_at": datetime.datetime.utcnow().isoformat() + "Z",
    "xgboost_version": xgb.__version__,
}

with open(save_dir / "models_meta.json", "w") as f:
    json.dump(models_meta, f, indent=2)
print("Wrote:", save_dir / "models_meta.json")

# sku_tier_map.csv & sku_tier_map.json (SKU → tier used)
# Expect sku_tier: pd.Series where index=SKU, value=tier
sku_tier_path_csv = save_dir / "sku_tier_map.csv"
sku_tier_path_json = save_dir / "sku_tier_map.json"

sku_tier.to_frame().reset_index().rename(columns={"index": "SKU"}).to_csv(sku_tier_path_csv, index=False)
sku_tier.to_frame().reset_index().rename(columns={"index": "SKU"}).to_json(sku_tier_path_json, orient="records")
print("Wrote:", sku_tier_path_csv)
print("Wrote:", sku_tier_path_json)

# human-readable report
pd.DataFrame(report_rows).sort_values("tier").to_csv(save_dir / "models_report.csv", index=False)
print("Wrote:", save_dir / "models_report.csv")

Wrote: /Users/joelsng/Documents/GitHub/levelsliving-IMS/server/forecasting/models/models_meta.json
Wrote: /Users/joelsng/Documents/GitHub/levelsliving-IMS/server/forecasting/models/sku_tier_map.csv
Wrote: /Users/joelsng/Documents/GitHub/levelsliving-IMS/server/forecasting/models/sku_tier_map.json
Wrote: /Users/joelsng/Documents/GitHub/levelsliving-IMS/server/forecasting/models/models_report.csv


In [206]:
inventory_monthly.columns

Index(['Month', 'Quantity', 'lag_1', 'lag_2', 'lag_3', 'month_num', 'year',
       'month_sin', 'month_cos', 'log_qty'],
      dtype='object')

In [207]:
import json

# Ensure Month is datetime
inventory_monthly["Month"] = pd.to_datetime(inventory_monthly["Month"], errors="coerce")

# Sort by date so frontend displays correctly
df_sorted = inventory_monthly.sort_values("Month")

# Format for JS copy-paste
records = [
    {
        "date": d.strftime("%Y-%m-%d"),
        "quantity": int(q) if pd.notna(q) else None,
    }
    for d, q in zip(df_sorted["Month"], df_sorted["Quantity"])
]

js_array = "const chartDataBase = " + json.dumps(records, indent=2) + ";"
print(js_array)


const chartDataBase = [
  {
    "date": "2022-04-30",
    "quantity": 91
  },
  {
    "date": "2022-05-31",
    "quantity": 280
  },
  {
    "date": "2022-06-30",
    "quantity": 339
  },
  {
    "date": "2022-07-31",
    "quantity": 431
  },
  {
    "date": "2022-08-31",
    "quantity": 392
  },
  {
    "date": "2022-09-30",
    "quantity": 534
  },
  {
    "date": "2022-10-31",
    "quantity": 412
  },
  {
    "date": "2022-11-30",
    "quantity": 704
  },
  {
    "date": "2022-12-31",
    "quantity": 720
  },
  {
    "date": "2023-01-31",
    "quantity": 502
  },
  {
    "date": "2023-02-28",
    "quantity": 470
  },
  {
    "date": "2023-03-31",
    "quantity": 391
  },
  {
    "date": "2023-04-30",
    "quantity": 417
  },
  {
    "date": "2023-05-31",
    "quantity": 346
  },
  {
    "date": "2023-06-30",
    "quantity": 495
  },
  {
    "date": "2023-07-31",
    "quantity": 896
  },
  {
    "date": "2023-08-31",
    "quantity": 834
  },
  {
    "date": "2023-09-30",
    "quantit